# fNIRS to fMRI
In this notebook, we develop a pipeline for turning fMRI data into pseudo-fNIRS data, and we apply it to the NSD.

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

from subject import Subject
import utils
import pmcx

## Install Libraries

In [3]:
## uncomment if FSL library not installed
# !sudo apt -qq install file
# !wget https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py
# !python3 fslinstaller.py

from pathlib import Path
fslpath = Path(os.getcwd()).parents[1].as_posix()+'/fsl'
os.environ["FSLDIR"] = fslpath
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"
os.environ["PATH"] += os.pathsep + os.path.join(fslpath, 'bin')
!. ${FSLDIR}/etc/fslconf/fsl.sh

## Pipeline

In [17]:
# define subject (id: '01' to '08') and create segmentation
id = '01'
subj = Subject(id=id)

In [18]:
# place optodes and visualize in anatomical space
subj.place_optodes(nsources=10, ndetectors=100)
# subj.display_setup()

/home/thomasribeiro/.local/share/virtualenvs/tomography-experiments-ii9qT8Yt/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Probes placed... Bringing toward center in by 3%


/home/thomasribeiro/.local/share/virtualenvs/tomography-experiments-ii9qT8Yt/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Probes placed... Bringing toward center in by 3%


In [19]:
# transform anatomical segmentation into a given session (id: '21' to '30') and run (id: '01' to '12')
sessionID = '21'; runID = '01'
seg_transformed, geom_transformed = subj.transform(sessionID=sessionID, runID=runID)
subj.display_setup(seg_transformed, geom_transformed)

Saved to test.json


In [20]:
# create time-varying optical medium
media_properties = [
    [0,       0,       1.0000,  1.0000], # background/air (not air pockets!)
    [0.0190,  7.8182,  0.8900,  1.3700], # scalp/skull
    [0.0040,  0.0090,  0.8900,  1.3700], # csf
    [0.0200,  9.0000,  0.8900,  1.3700], # gray matters
    [0.0800,  40.9000, 0.8400,  1.3700]] # white matters

fmri = nib.load(f'data/sub{subj.id}/func/fmri/sess{sessionID}/run{runID}/sub-{subj.id}_ses-nsd{sessionID}_task-nsdcore_run-{runID}_bold.nii.gz').get_fdata()
optical_vol, optical_baseline = utils.fmri2optical(fmri, seg_transformed, media_properties)
# plt.imshow(optical_baseline[0,:,:,42]); plt.colorbar(); plt.show()
# plt.imshow(optical_vol[0,:,:,42,2]); plt.colorbar(); plt.show()

/home/thomasribeiro/tomography-experiments/fmri2fnirs/utils.py:396: RuntimeWarning:

invalid value encountered in divide



In [21]:
# run mcx simulation

n = 1.37 # index of refraction
g = 0.9  # anisotropy factor

# vol, prop = seg_transformed, media_properties       # this works
vol, prop = optical_vol[...,0], [[0,0,1,1],[0,0,g,n]] # this doesn't work

i = 0
cfg = {
       'nphoton': 1000000,
       'vol': vol,
       'tstart': 0,
       'tend': 1e-8,
       'tstep': 1e-10,
       'srcpos': geom_transformed.sources[i],
       'srcdir': geom_transformed.directions[i],
       'prop': prop,
    #    'maxdetphoton': 1000000000,
       }

res = pmcx.run(cfg)

nphoton: 1e+06
tstart: 0
tstep: 1e-10
tend: 1e-08
srcpos: [78.6103, 102.741, 58.3014, 1]
srcdir: [-0.416458, -0.850672, -0.32081, 0]


###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2023 Qianqian Fang <q.fang at neu.edu>          #
#                https://mcx.space/  &  https://neurojson.org/                #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
###############################################################################
#    The MCX Project is funded by the NIH/NIGMS under grant R01-GM114365      #
###############################################################################
#  Open-source codes and reusable scientific data are essential for research, #
# MCX proudly developed human-readable JSON-based data formats for easy reuse,#
#  Please consider using JSON (https://n

In [25]:
subj.display_setup(res['flux'][...,30], geom_transformed)

In [ ]:
data = utils.get_detector_data(res['flux'], geom_transformed.detectors, kernel_size=5)
plt.imshow(np.log10(data).T); plt.show()

In [19]:
# plt.hist(res['detp'][1], bins=100, range=[0,200])
# plt.show()